# 5 Урок: A/В-тесты

---

## Задание №3: Проведение A/B-теста

>**Задача:**
>
> 1) Проанализируйте тест между группами 0 и 3 по метрике линеаризованных лайков. Видно ли отличие? Стало ли 𝑝−𝑣𝑎𝑙𝑢𝑒 меньше?
>
> 2) Проанализируйте тест между группами 1 и 2 по метрике линеаризованных лайков. Видно ли отличие? Стало ли 𝑝−𝑣𝑎𝑙𝑢𝑒 меньше?

---
## Установим необходимые библиотеки и подключение к БД

In [1]:
# установим необходимые библиотеки

import pandas as pd
import numpy as np
import pandahouse as ph
from scipy import stats as st
import matplotlib.pyplot as plt
import seaborn as sns

In [45]:
import warnings
warnings.simplefilter(action='ignore')

In [2]:
# установим фиксированные размер для будущих графиков

sns.set(rc={'figure.figsize':(12,9)})

In [3]:
# установим соединение с БД

con = {
    'host': '*****',
    'password': '*****',
    'user': '*****',
    'database': '*****'
}

## Напишем запрос и вытащим наши данные

In [13]:
## напишем запрос

query = '''

SELECT exp_group,
       user_id,
       sum(action = 'view') AS view,
       sum(action = 'like') AS like,
       like / view AS ctr
FROM simulator_20230220.feed_actions       
WHERE (toDate(time) >= '2022-01-31' AND toDate(time) <='2023-02-06')
      AND exp_group IN (0, 1, 2, 3)
GROUP BY exp_group, user_id
ORDER BY exp_group, user_id, view, like, ctr


'''

In [14]:
# поместим данные в переменную df

df = ph.read_clickhouse(query, connection=con)

In [15]:
# выведем на экран

df.head()

,exp_group,user_id,view,like,ctr
0,0,201,208,59,0.283654
1,0,211,151,26,0.172185
2,0,212,348,94,0.270115
3,0,222,156,28,0.179487
4,0,225,41,8,0.195122


In [16]:
# сравник количество участников

df.groupby('exp_group').count()

,user_id,view,like,ctr
exp_group,,,,
0,13019,13019,13019,13019
1,13123,13123,13123,13123
2,12987,12987,12987,12987
3,13214,13214,13214,13214


Группы примерно равны

## Напишем функцию

Которая рассчитает общегрупповой CTR и выведет новую метрику **𝑙𝑖𝑛𝑒𝑎𝑟𝑖𝑧𝑒𝑑_𝑙𝑖𝑘𝑒𝑠**, а также при желании проведет сразу 2 Т-теста по CTR и по новой метрики.



In [99]:
# напишем функцию для расчета поюзерной метрики linearized_likes

def ctr_control(data, control_group, target_group, ttest=True):
    
    # сделаем срез по группам
    ctr_control = data[(data['exp_group'] == control_group) | (data['exp_group'] == target_group)]
    
    # расчитаем общий CTR в контрольной группе
    ctr_control['ctr_control'] = (
        ctr_control[ctr_control['exp_group'] == control_group]['like'].sum() 
           / ctr_control[ctr_control['exp_group'] == control_group]['view'].sum()
    )
    
    # Посчитаем в обеих группах поюзерную метрику  linearized_likes = like - CTRcontrol * view
    ctr_control['linearized_likes'] = ctr_control['like'] - (ctr_control['ctr_control'] * ctr_control['view'])
    
    
    
    # проведем Т-тест
    if ttest == True:
        print(f'Проведем t-тест для CTR в группах: Контрольная - {control_group} и Целевая - {target_group}\n\n')
        result = (
            st.ttest_ind(ctr_control[ctr_control.exp_group == control_group].ctr, 
                        ctr_control[ctr_control.exp_group == target_group].ctr,
                         equal_var = True # специально выделил, что будем считать дисперсии равными                        
                        )
        )
            # статистическую значимость возьмем в 5%
        alpha = .05 
        if result.pvalue < alpha:
            print('Отвергаем нулевую гипотезу. Между данными есть статистически значимая разница')
        else:
            print('Не получилость отвергнуть нулевую гипотезу. Между данными не удалось найти статистически значимую разницу')
        print(result)    
        
        print(f'\n\nПроведем t-тест для поюзерной метрики linearized_likes в группах: Контрольная - {control_group} и Целевая - {target_group}\n\n')
        result_2 = (
            st.ttest_ind(ctr_control[ctr_control.exp_group == control_group].linearized_likes, 
                        ctr_control[ctr_control.exp_group == target_group].linearized_likes,
                         equal_var = True # специально выделил, что будем считать дисперсии равными                        
                        )
        )
            # статистическую значимость возьмем в 5%
        alpha = .05 
        if result_2.pvalue < alpha:
            print('Отвергаем нулевую гипотезу. Между данными есть статистически значимая разница')
        else:
            print('Не получилость отвергнуть нулевую гипотезу. Между данными не удалось найти статистически значимую разницу')
        print(result_2)    
        
        # на всякий случай пусть наша функцию выведет таблицу с нашей новой метрикой, вдруг вдальнейшем понадобится
        return ctr_control
            
            

## Проведем t-тест для групп 0 и 3

In [100]:
# применим нашу функцию и сразу проведет т-тест

group_03 = ctr_control(df, control_group=0, target_group=3, ttest=True)

Проведем t-тест для CTR в группах: Контрольная - 0 и Целевая - 3


Отвергаем нулевую гипотезу. Между данными есть статистически значимая разница
Ttest_indResult(statistic=-8.05088595006658, pvalue=8.56540958071733e-16)


Проведем t-тест для поюзерной метрики linearized_likes в группах: Контрольная - 0 и Целевая - 3


Отвергаем нулевую гипотезу. Между данными есть статистически значимая разница
Ttest_indResult(statistic=-6.585317270787503, pvalue=4.624933110497003e-11)


**Итог теста:**
- в обоих случаях мы **отвергли нулевую гипотезу** и обнаружили между данными статистически значимую разницу;
- стоит отметить, что при проведении т-тест с учетом нашей новой метрики linearized_likes значение p-value уменьшилось.

## Проведем t-тест для групп 1 и 2

In [101]:
# применим нашу функцию и сразу проведет т-тест

group_03 = ctr_control(df, control_group=1, target_group=2, ttest=True)

Проведем t-тест для CTR в группах: Контрольная - 1 и Целевая - 2


Не получилость отвергнуть нулевую гипотезу. Между данными не удалось найти статистически значимую разницу
Ttest_indResult(statistic=1.5009629042115853, pvalue=0.13337724953264857)


Проведем t-тест для поюзерной метрики linearized_likes в группах: Контрольная - 1 и Целевая - 2


Отвергаем нулевую гипотезу. Между данными есть статистически значимая разница
Ttest_indResult(statistic=2.066277941426868, pvalue=0.038812082333477466)


**Итог теста:**
- при проведении теста в первом случай (при сравнении CTR) у нас не получилось отвергнуть нулевую гипотезу и уровень p-value около 13%, а вот при расчете нашей новой метрики linearized_likes уровень p-value упал до примерно 3% и мы уже отвергаем нуевую гипотезу.